# Autonomy fleet visualization

The objectives of this notebook are to :
<ol>
    <li> Download autonomy assets data </li>
    <li> Show the current location of autonomy assets </li>
    <li> Show the last profiles of the assets </li>
    <li> Show the full depth/time transect of the assets </li>
</ol>

To run this notebook please refer to the readme of the github page (https://github.com/NOC-OBG-Autonomy/biocarbon_nrt_data_viz).

## Download autonomy assets data

In [1]:
import pandas as pd
import numpy as np
import requests
import shutil
import os
import gzip
from pathlib import Path
import re
from tqdm import tqdm
from urllib.request import urlretrieve
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from datetime import datetime
from scipy import interpolate
import warnings
warnings.filterwarnings('ignore')

We build the data repository structure. The data folder is not sync with git, so we need to create it if missing (i.e. first time your run this notebook on your computer).

In [2]:
def create_missing_directories():
    # Define the path to the parent directory
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

    # Check if 'data' folder exists in the parent directory
    data_dir = os.path.join(parent_dir, 'data')
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print("'data' folder created in the parent directory.")
    else:
        print("'data' folder already exists in the parent directory.")

    # Check if 'floats' directory exists inside 'data' folder
    floats_dir = os.path.join(data_dir, 'Floats')
    if not os.path.exists(floats_dir):
        os.makedirs(floats_dir)
        print("'floats' directory created inside 'data' folder.")
    else:
        print("'floats' directory already exists inside 'data' folder.")

    # Check if 'gliders' directory exists inside 'data' folder
    gliders_dir = os.path.join(data_dir, 'Gliders')
    if not os.path.exists(gliders_dir):
        os.makedirs(gliders_dir)
        print("'gliders' directory created inside 'data' folder.")
    else:
        print("'gliders' directory already exists inside 'data' folder.")

if __name__ == "__main__":
    create_missing_directories()

'data' folder already exists in the parent directory.
'floats' directory already exists inside 'data' folder.
'gliders' directory already exists inside 'data' folder.


For now, we will visualize only test floats. One in the Icelandic bassin and the other in SO during the Custard cruise, which is a good test case as it has also gliders deployements.

We don't automate in this script the retrieving of downloading url for the sake of simplicity. 
You can keep in mind that the list of float data and their repertory are findable here : https://data-argo.ifremer.fr/argo_synthetic-profile_index.tx

In [3]:
wmo_list = [6990636, 4903659, 3901581, 7902223, 3902261]
#Float 1 = Flux1 float LOV
float_1_url = 'https://data-argo.ifremer.fr/dac/coriolis/6990636/6990636_Sprof.nc'
#Float 2 = Flux2 float LOV
float_2_url = 'https://data-argo.ifremer.fr/dac/coriolis/4903659/4903659_Sprof.nc'
#Float 3 = test float on Custard with glider next to it
float_3_url = 'https://data-argo.ifremer.fr/dac/bodc/3901581/3901581_Sprof.nc'
#Float 4 = test float on Custard with glider next to it
float_4_url = 'https://data-argo.ifremer.fr/dac/coriolis/7902223/7902223_Sprof.nc'
#Float 5 nearest to CIB
float_5_url = 'https://data-argo.ifremer.fr/dac/aoml/3902261/3902261_Sprof.nc'

#List the floats to plot
floats_url = [float_1_url, float_2_url, float_3_url, float_4_url, float_5_url]

#Assign the local float directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
floats_dir =  os.path.join(parent_dir, 'Data\\Floats')

#Create floats filename
floats_filenames = []
for i in floats_url:
    filename = floats_dir + '/' + i.rsplit('/', 1)[1]
    floats_filenames.append(filename)

In [4]:
#Download floats data. We do it everytime so we are sure to work with a DAC up to date version. 
#We could check with the synthetic profile index if it is needed to download a update.
for url, filename in zip(floats_url, floats_filenames):
    urlretrieve(url, filename)


In [ ]:
filenames = []
for i in range(14):
    url = 'https://data-argo.ifremer.fr/dac/bodc/3901581/profiles/SR3901581_0' + format(i + 1, '02d') + '.nc'
    filename = floats_dir + '/' + url.rsplit('/', 1)[1]
    urlretrieve(url, filename)
    filenames.append(filename)

In [ ]:
uk_position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})
for file in filenames:
    dat = xr.open_dataset(file)
    dat = dat.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})
    temp_df = dat[['LONGITUDE', 'LATITUDE', 'JULD']].to_dataframe().reset_index()
    temp_df['float'] = 3901581
    uk_position_df = pd.concat([uk_position_df, temp_df], ignore_index=True)
    dat.close()
    

In [ ]:
uk_position_df = uk_position_df.drop_duplicates()

## Plot BGC-Argo locations

### Icelandic Bassin float

There are 2 floats near the icelandic bassin. One on the north west that is currently leaving the bassin by the ridge and one that is coming by the south (deployed during APERO). For the second one there are some issues with the DAC so we are waiting for the data. 

In [ ]:
position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})
for file, wmo in zip(floats_filenames, wmo_list):
    dat = xr.open_dataset(file)
    dat = dat.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})
    temp_df = dat[['LONGITUDE', 'LATITUDE', 'JULD']].to_dataframe().reset_index()
    temp_df['float'] = wmo
    position_df = pd.concat([position_df, temp_df], ignore_index=True)
    dat.close()

In [ ]:
other_df = pd.read_csv(parent_dir + '/Plotting_tools/shared_data/rt_positions.csv')

In [ ]:
float_df = other_df[other_df['platform_id'] == 'lovuse031c']

In [ ]:
missing_float = pd.DataFrame({'PROF_NUM' : float_df['Unnamed: 0'], 'LONGITUDE' : float_df['lon'], 'LATITUDE' : float_df['lat'], 'float' : 1902695, 'JULD' : pd.to_datetime(float_df['date'])})

In [ ]:
tables = [position_df, missing_float]
position_df = pd.concat(tables)
position_df = pd.concat([position_df, uk_position_df])

In [ ]:
import cartopy.feature as cfeature
import cartopy.crs as ccrs

land_50m = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='k',
                                        facecolor=cfeature.COLORS['land'])

# Define data's extents I used an arbitrary extent that depicts the Icelandic Bassin
min_lon = -33
max_lon = -13
min_lat = 55
max_lat = 65


In [ ]:
bath_directory = parent_dir + '/Data/' + "ne_10m_bathymetry_all/"
def load_bathymetry(zip_file_url):
    """Read zip file from Natural Earth containing bathymetry shapefiles"""
    # Download and extract shapefiles
    import io
    import zipfile

    #r = requests.get(zip_file_url)
    #z = zipfile.ZipFile(io.BytesIO(r.content))
    #z.extractall(bath_directory)

    # Read shapefiles, sorted by depth
    shp_dict = {}
    files = glob(bath_directory + '*.shp')
    assert len(files) > 0
    files.sort()
    depths = []
    for f in files:
        depth = '-' + f.split('_')[-1].split('.')[0]  # depth from file name
        depths.append(depth)
        bbox = (min_lon - 3, max_lon + 3,min_lat - 1, max_lat + 1)  # (x0, y0, x1, y1)
        nei = shpreader.Reader(f, bbox=bbox)
        shp_dict[depth] = nei
    depths = np.array(depths)[::-1]  # sort from surface to bottom
    return depths, shp_dict

In [ ]:
import cartopy.io.shapereader as shpreader
depths_str, shp_dict = load_bathymetry(
        'https://naturalearth.s3.amazonaws.com/' +
        '10m_physical/ne_10m_bathymetry_all.zip')


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib
position_df = position_df.sort_values(by=['JULD'])
grouped = position_df.groupby('float')

# Construct a discrete colormap with colors corresponding to each depth
depths = depths_str.astype(int)
N = len(depths)
nudge = 0.01  # shift bin edge slightly to include data
boundaries = [min(depths)] + sorted(depths+nudge)  # low to high
norm = matplotlib.colors.BoundaryNorm(boundaries, N)
blues_cm = matplotlib.colormaps['Blues_r'].resampled(N)
colors_depths = blues_cm(norm(depths))

# Set up plot
# Initialize an empty figure and add an axis
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1,
                    projection=ccrs.Mercator())

# Set the map extent based on your latitude and longitude ranges
ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())

# Scatter plot
sc = ax.scatter(position_df['LONGITUDE'], position_df['LATITUDE'], transform=ccrs.PlateCarree(), c = position_df['JULD'], zorder = 3)

#set the plot color bar
cbar = plt.colorbar(sc, ax = ax, label='Date')
cbar.set_label('Date', rotation=270, labelpad=15)

float_array = cbar.ax.get_yticks()
formatted_date = np.vectorize(lambda x: datetime.fromtimestamp(float(x) / 1e9).strftime("%b %Y"))(float_array)
cbar.ax.set_yticklabels(formatted_date)

for i, depth_str in enumerate(depths_str):
    ax.add_geometries(shp_dict[depth_str].geometries(),
                        crs=ccrs.PlateCarree(),
                        color=colors_depths[i])

for name, group in grouped:
    group.plot(x='LONGITUDE', y='LATITUDE', ax=ax, transform=ccrs.PlateCarree(), label=name, zorder=2)
# Add map features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(land_50m)
ax.stock_img()

# print a grid on it
gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())

# Convert vector bathymetries to raster (saves a lot of disk space)
# while leaving labels as vectors
ax.set_rasterized(True)


In [ ]:
position_df.to_csv(parent_dir + '/Data/Floats/floats_location.csv')

Making a csv data

In [ ]:
wmo_list = [4903532, 1902637, 6990636, 3901581]
float_files = [ f for f in os.listdir(parent_dir + '/Data/Floats') if any(str(wmo) in f for wmo in wmo_list)] 

In [ ]:
data_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [],'float' : int()})
for file in float_files:
    wmo = re.findall(r'\d{7}', file)[0]
    dat = xr.open_dataset(parent_dir + '/Data/Floats/' + file)
    dat = dat.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})
    if 'NITRATE_ADJUSTED' in dat.variables:
        temp_df = dat[['LONGITUDE', 'LATITUDE', 'JULD', 'PRES', 'CHLA_ADJUSTED', 'PSAL_ADJUSTED', 'TEMP_ADJUSTED', 'NITRATE_ADJUSTED']].to_dataframe().reset_index()
        temp_df['float'] = wmo
        data_df = pd.concat([data_df, temp_df], ignore_index=True)
        dat.close()
    else :
        print(f'{wmo} floats has no Nitrate')
        temp_df = dat[['LONGITUDE', 'LATITUDE', 'JULD', 'PRES', 'CHLA_ADJUSTED', 'PSAL_ADJUSTED', 'TEMP_ADJUSTED']].to_dataframe().reset_index()
        temp_df['float'] = wmo
        data_df = pd.concat([data_df, temp_df], ignore_index=True)
        dat.close()

In [ ]:
data_df.to_csv(parent_dir + '/Data/Floats/floats_nitrate.csv')

## Profile plotting

In [ ]:
dat = xr.open_dataset(floats_filenames[1])
df = dat[['JULD', 'PRES', 'TEMP', 'PSAL', 'CHLA_ADJUSTED', 'BBP700_ADJUSTED', 'DOXY_ADJUSTED']].to_dataframe()
dat.close()
df = df.reset_index().set_index('JULD', drop=False)

In [ ]:
import math
last_date = max(df['JULD'])

last_df = df[df['JULD'] == last_date]
early_df = df[df['JULD'] != last_date]
early_df = early_df[early_df['JULD'] > pd.to_datetime('2024-01-01')]

alphas = (early_df['N_PROF'] - min(early_df['N_PROF']))/(max(early_df['N_PROF']) - min(early_df['N_PROF']))

## Profiles

### Temperature
The temperature profiles. Last profile is in black, past float profiles are in grey

In [ ]:
lastmonth_df = df[df['JULD'] > pd.to_datetime('2024-03-25')]
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

sc2 = ax.scatter( early_df['TEMP'], - (early_df['PRES']), alpha = alphas/5, c = 'grey')
sc = ax.scatter( last_df['TEMP'], - (last_df['PRES']), c = 'black')

#set the plot color bar
#cbar = plt.colorbar(sc, ax = ax, label='Julian day')
#cbar.set_label('Julian Day', rotation=270, labelpad=15)
ax.set_ylim([-1000,0])

ax.set_xlabel('Degrees celcius')
ax.set_ylabel('Depth')
# set the plot title
ax.set_title('Float wmo : ' + str(wmo) + "\n Temperature profile : " + last_date.strftime("%Y-%m-%d %H:%M:%S"))
plt.show()

Zoom in the upper 500m during the last month

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

sc = ax.scatter( lastmonth_df['TEMP'], - (lastmonth_df['PRES']), c = lastmonth_df['JULD'])

#set the plot color bar
cbar = plt.colorbar(sc, ax = ax, label='Julian day')
cbar.set_label('Date', rotation=270, labelpad=15)
float_array = cbar.ax.get_yticks()
formatted_date = np.vectorize(lambda x: datetime.fromtimestamp(float(x) / 1e9).strftime("%d %b %Y"))(float_array)
cbar.ax.set_yticklabels(formatted_date)
ax.set_ylim([-500,0])
ax.set_xlim([6.5,12])

ax.set_xlabel('Degrees celcius')
ax.set_ylabel('Depth (m)')
# set the plot title
ax.set_title('Float wmo : ' + str(wmo) + "\n Temperature profile : " + last_date.strftime("%Y-%m-%d %H:%M:%S"))
plt.show()

### Chlorophyll a
The chla profiles. Last profile is in black, past float profiles are in grey

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

sc2 = ax.scatter( early_df['CHLA_ADJUSTED'], - (early_df['PRES']), alpha = alphas, c = 'grey')
sc = ax.scatter( last_df['CHLA_ADJUSTED'], - (last_df['PRES']), c = 'black')

#set the plot color bar
#cbar = plt.colorbar(sc, ax = ax, label='Julian day')
#cbar.set_label('Julian Day', rotation=270, labelpad=15)
ax.set_ylim([-1000,0])
ax.set_xlim([0,1.5])

ax.set_xlabel('Chla (mg.m-3)')
ax.set_ylabel('Depth (m)')
# set the plot title
ax.set_title('Float wmo : ' + str(wmo) + "\n Chlorophyll-a profile : " + last_date.strftime("%Y-%m-%d %H:%M:%S"))
plt.show()

Zoom in the upper 500m during the last month

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

sc = ax.scatter( lastmonth_df['CHLA_ADJUSTED'], - (lastmonth_df['PRES']), c = lastmonth_df['JULD'])

#set the plot color bar
cbar = plt.colorbar(sc, ax = ax, label='Julian day')
cbar.set_label('Date', rotation=270, labelpad=15)
float_array = cbar.ax.get_yticks()
formatted_date = np.vectorize(lambda x: datetime.fromtimestamp(float(x) / 1e9).strftime("%d %b %Y"))(float_array)
cbar.ax.set_yticklabels(formatted_date)
ax.set_ylim([-500,0])
ax.set_xlim([0,2])

ax.set_xlabel('Chla (mg.m-3)')
ax.set_ylabel('Depth (m)')
# set the plot title
ax.set_title('Float wmo : ' + str(wmo) + "\n Chlorophyll-a profile : " + last_date.strftime("%Y-%m-%d %H:%M:%S"))
plt.show()

### BBP700 profiles

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

sc2 = ax.scatter( early_df['BBP700_ADJUSTED'], - (early_df['PRES']), alpha = alphas/5, c = 'grey')
sc = ax.plot( last_df['BBP700_ADJUSTED'], - (last_df['PRES']), c = 'black')

#set the plot color bar
#cbar = plt.colorbar(sc, ax = ax, label='Julian day')
#cbar.set_label('Julian Day', rotation=270, labelpad=15)
ax.set_ylim([-1000,0])
ax.set_xlim([0.0002,0.01])

ax.set_xlabel('bbp (m-1)')
ax.set_ylabel('Depth (m)')
# set the plot title
ax.set_title('Float wmo : ' + str(wmo) + "\n BBP 700 profile : " + last_date.strftime("%Y-%m-%d %H:%M:%S"))
plt.show()

Zoom in the upper 500m during the last month.

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

sc = ax.scatter( lastmonth_df['BBP700_ADJUSTED'], - (lastmonth_df['PRES']), c = lastmonth_df['JULD'])

#set the plot color bar
cbar = plt.colorbar(sc, ax = ax, label='Julian day')
cbar.set_label('Date', rotation=270, labelpad=15)
float_array = cbar.ax.get_yticks()
formatted_date = np.vectorize(lambda x: datetime.fromtimestamp(float(x) / 1e9).strftime("%d %b %Y"))(float_array)
cbar.ax.set_yticklabels(formatted_date)
ax.set_ylim([-500,0])

ax.set_xlabel('Bbp 700 (m-1)')
ax.set_ylabel('Depth (m)')
# set the plot title
ax.set_title('Float wmo : ' + str(wmo) + "\n BBP700 profile : " + last_date.strftime("%Y-%m-%d %H:%M:%S"))
plt.show()

Timeseries of the ratio between bbp and Chla, in the first 5m

In [ ]:
timeseries_df = df[df['JULD'] > pd.to_datetime('2023-05-01')]
timeseries_df = timeseries_df[timeseries_df['PRES'] < 5]

In [ ]:
ratio = timeseries_df['BBP700_ADJUSTED'] / timeseries_df['CHLA_ADJUSTED']

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

sc2 = ax.scatter( timeseries_df['JULD'], ratio)

#set the plot color bar
#cbar = plt.colorbar(sc, ax = ax, label='Julian day')
#cbar.set_label('Julian Day', rotation=270, labelpad=15)

ax.set_xlabel('Date')
ax.set_ylabel('BBP/Chla')
# set the plot title
ax.set_ylim([0, 0.01])
ax.set_title('Float wmo : ' + str(wmo) + "\n BBP/Chla time series : " + last_date.strftime("%Y-%m-%d %H:%M:%S"))
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

sc2 = ax.scatter( early_df['DOXY_ADJUSTED'], - (early_df['PRES']), alpha = alphas/5, c = 'grey')
sc = ax.scatter( last_df['DOXY_ADJUSTED'], - (last_df['PRES']), c = 'black')

#set the plot color bar
#cbar = plt.colorbar(sc, ax = ax, label='Julian day')
#cbar.set_label('Julian Day', rotation=270, labelpad=15)
ax.set_ylim([-1000,0])

ax.set_xlabel('Dissolved Oxygen (umol.kg-1)')
ax.set_ylabel('Depth (m)')
# set the plot title
ax.set_title('Float wmo : ' + str(wmo) + "\n Dissolved oxygen profile : " + last_date.strftime("%Y-%m-%d %H:%M:%S"))
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot()

sc = ax.scatter( lastmonth_df['DOXY_ADJUSTED'], - (lastmonth_df['PRES']), c = lastmonth_df['JULD'])

#set the plot color bar
cbar = plt.colorbar(sc, ax = ax, label='Julian day')
cbar.set_label('Date', rotation=270, labelpad=15)
float_array = cbar.ax.get_yticks()
formatted_date = np.vectorize(lambda x: datetime.fromtimestamp(float(x) / 1e9).strftime("%d %b %Y"))(float_array)
cbar.ax.set_yticklabels(formatted_date)
ax.set_ylim([-500,0])

ax.set_xlabel('Dissolved oxygen (umol.kg-1)')
ax.set_ylabel('Depth (m)')
# set the plot title
ax.set_title('Float wmo : ' + str(wmo) + "\n Dissolved oxygen profile : " + last_date.strftime("%Y-%m-%d %H:%M:%S"))
plt.show()

## Transect of the float

Some time series visualisation of the float from 2024. 

In [ ]:
data = xr.open_dataset(floats_filenames[0])
data = data.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})

In [ ]:
# Stack NumPy array of datetimes to create a 2D grid
time_2D = np.tile(data['JULD'].values,(len(data['N_LEVELS']),1)).T

# Function for repetitive parts of plot
def config_depth_section(cbar_label,title):
  plt.ylim([0,250])
  plt.gca().invert_yaxis()
  plt.ylabel('Pressure (dbar)')
  if 'Nitrate' in cbar_label: extend = 'min'
  else:                       extend = 'neither'
  plt.colorbar(label=cbar_label,extend=extend)
  plt.title(title)

In [ ]:
# Function to convert an array from NumPy datetime64 to Python float format
def datetime_to_float(dt):
  return (dt - np.datetime64('1900-01-01')) / np.timedelta64(1,'D')

# Function to convert an array from Python float to NumPy datetime64 format
def float_to_datetime(nums):
  return (nums * np.timedelta64(1,'D')) + np.datetime64('1900-01-01')

# Function to interpolate data from a specified float parameter to a uniform time and pressure grid
def interpolate_depth_section(param_name,specify_qc_flags=None,pres_interval=1.0):
  """
  Arguments:
      param_name: string with netCDF file parameter name (e.g., 'TEMP_ADJUSTED') to interpolate
      specify_qc_flags: None to ignore QC flags
                        or a list of QC flags (e.g., [1,2,3]) indicating which data to retain before interpolation
      pres_interval: vertical resolution for interpolating pressure (z) axis (default: 1.0 dbar)
  
  Returns:
      time_coord: 1-D NumPy array with original profile timestamps in np.datetime64 format
      pres_coord: 1-D NumPy array with a uniform pressure (z) coordinate from 0 dbar to the deepest recorded
                  pressure value, at a resolution of <pres_interval> dbar
      time_grid: 2-D NumPy array with the meshed grid of time_coord
      pres_grid: 2-D NumPy array with the meshed grid of pres_coord
      param_gridded: 2-D NumPy array with the interpolated parameter values at the locations of time_grid and pres_grid

  """

  # New grid points
  time_coord = data['JULD'].values
  time_coord = time_coord[(time_coord >= np.datetime64('2023-10-01'))]
  pres_coord = np.arange(0,data['PRES'].max(),pres_interval)
  time_grid, pres_grid = np.meshgrid(time_coord,pres_coord)
  time_grid = datetime_to_float(time_grid)     # Convert from np.datetime64 to float

  # 1-D (flattened) versions of old grids and parameter values
  time_1D = np.tile(data['JULD'].values,(len(data['N_LEVELS']),1)).T.flatten()
  pres_1D = data['PRES'].values.flatten()
  param_1D = data[param_name].values.flatten()
  if param_1D.dtype == object:         # If parameter is an array of QC flag data
    param_1D = param_1D.astype(float)  # Convert QC flags from dtype 'object' to float
    interp_method = 'nearest'          # Use nearest-neighbor interpolation for QC flags to avoid unwanted averaging
  else:
    interp_method = 'linear'           # Use bilinear interpolation for normal data fields

  # Extract only values matching specified QC flags
  if specify_qc_flags is not None:
    qc_1D = data[param_name + '_QC'].values.astype(float).flatten()
    qc_mask = np.tile(False,len(qc_1D))
    for qc_flag in specify_qc_flags:
      qc_mask = np.logical_or(qc_mask,qc_1D == qc_flag)
    time_1D = time_1D[qc_mask]
    pres_1D = pres_1D[qc_mask]
    param_1D = param_1D[qc_mask]

  # Remove NaN values before interpolation
  time_1D = datetime_to_float(time_1D[~np.isnan(param_1D)])       # Convert from np.datetime64 to float
  pres_1D = pres_1D[~np.isnan(param_1D)]
  param_1D = param_1D[~np.isnan(param_1D)]

  # Interpolate from irregular points to grid
  param_gridded = interpolate.griddata((time_1D,pres_1D),param_1D,(time_grid,pres_grid),method=interp_method)

  # Return coordinates, grid, and gridded data
  return time_coord, pres_coord, float_to_datetime(time_grid), pres_grid, param_gridded

In [ ]:
# Interpolate data
param_name = 'BBP700_ADJUSTED'
time_coord, pres_coord, time_grid, pres_grid, param_gridded = interpolate_depth_section(param_name)

# Plot depth section of upper ocean only
plt.figure(figsize=(12,6))
plt.pcolormesh(time_grid,pres_grid,param_gridded)
plt.ylim([0,1000])
plt.gca().invert_yaxis()
plt.ylabel('Pressure (dbar)')
plt.colorbar(label='{0} ({1})'.format(data[param_name].long_name,data[param_name].units))
plt.title(data[param_name].long_name)

In [ ]:
# Interpolate data
param_name = 'TEMP_ADJUSTED'
time_coord, pres_coord, time_grid, pres_grid, param_gridded = interpolate_depth_section(param_name)

# Plot depth section of upper ocean only
plt.figure(figsize=(12,6))
plt.pcolormesh(time_grid,pres_grid,param_gridded)
plt.ylim([0,1000])
plt.gca().invert_yaxis()
plt.ylabel('Pressure (dbar)')
plt.colorbar(label='{0} ({1})'.format(data[param_name].long_name,data[param_name].units))
plt.title(data[param_name].long_name)

In [ ]:
# Interpolate data
param_name = 'CHLA_ADJUSTED'
time_coord, pres_coord, time_grid, pres_grid, param_gridded = interpolate_depth_section(param_name)

# Plot depth section of upper ocean only
plt.figure(figsize=(12,6))
plt.pcolormesh(time_grid,pres_grid,param_gridded)
plt.ylim([0,1000])
plt.gca().invert_yaxis()
plt.ylabel('Pressure (dbar)')
plt.colorbar(label='{0} ({1})'.format(data[param_name].long_name,data[param_name].units))
plt.title(data[param_name].long_name)

In [ ]:
# Interpolate data
param_name = 'DOXY_ADJUSTED'
time_coord, pres_coord, time_grid, pres_grid, param_gridded = interpolate_depth_section(param_name)

# Plot depth section of upper ocean only
plt.figure(figsize=(12,6))
plt.pcolormesh(time_grid,pres_grid,param_gridded)
plt.ylim([0,1000])
plt.gca().invert_yaxis()
plt.ylabel('Pressure (dbar)')
plt.colorbar(label='{0} ({1})'.format(data[param_name].long_name,data[param_name].units))
plt.title(data[param_name].long_name)